# Simple Prompt Engineering Types Tutorial

## A Beginner's Guide to Different Prompt Engineering Techniques using LangChain

This tutorial provides clear, simple examples of different prompt engineering types and techniques. Perfect for beginners who want to understand the fundamentals before diving into complex applications.

---

## Table of Contents
1. [Prompt Engineering Fundamentals](#1-prompt-engineering-fundamentals)
2. [Core Prompt Engineering Types](#2-core-prompt-engineering-types)
3. [Simple Practical Examples](#3-simple-practical-examples)
4. [Prompt Optimization Techniques](#4-prompt-optimization-techniques)
5. [Common Patterns and Templates](#5-common-patterns-and-templates)
6. [Interactive Examples with LangChain](#6-interactive-examples-with-langchain)

---

## Setup and Dependencies

In [ ]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
import json
from typing import List, Dict

# Load environment variables
load_dotenv(dotenv_path="/home/bipin/Documents/genai/g25-aug/genai_25_sept/.env")

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini", max_completion_tokens=500, temperature=0.3)

---

## 1. Prompt Engineering Fundamentals

### What is Prompt Engineering?

Prompt engineering is the art and science of crafting effective instructions for AI language models. Think of it as learning how to communicate clearly with a very smart but literal assistant.

### Why Does It Matter?
- **Better Results**: Well-crafted prompts produce more accurate and useful responses
- **Consistency**: Good prompts give consistent results across multiple uses
- **Efficiency**: Save time by getting the right answer on the first try
- **Control**: Guide the AI's behavior and output format

### Basic Principles

#### 1. Be Clear and Specific

In [ ]:
# Bad prompt: Vague and unclear
bad_prompt = ChatPromptTemplate.from_messages([
    ("human", "Tell me about dogs")
])

# Good prompt: Clear and specific
good_prompt = ChatPromptTemplate.from_messages([
    ("human", "List 5 popular dog breeds suitable for apartment living, including their size and key characteristics")
])

# Test both prompts
print("=== BAD PROMPT (Vague) ===")
bad_chain = bad_prompt | llm
bad_response = bad_chain.invoke({})
print(bad_response.content[:200] + "...\n")

print("=== GOOD PROMPT (Specific) ===")
good_chain = good_prompt | llm
good_response = good_chain.invoke({})
print(good_response.content)

#### 2. Provide Context When Needed

In [ ]:
# Without context
no_context_prompt = ChatPromptTemplate.from_messages([
    ("human", "Should I invest in this stock?")
])

# With context
with_context_prompt = ChatPromptTemplate.from_messages([
    ("human", """I'm a 25-year-old software engineer with a stable income and $10,000 to invest. 
    I have a moderate risk tolerance and a 10-year investment timeline. 
    Should I invest in Apple stock (AAPL) which is currently trading at $150 with a P/E ratio of 25?""")
])

print("=== WITHOUT CONTEXT ===")
no_context_chain = no_context_prompt | llm
no_context_response = no_context_chain.invoke({})
print(no_context_response.content[:200] + "...\n")

print("=== WITH CONTEXT ===")
with_context_chain = with_context_prompt | llm
with_context_response = with_context_chain.invoke({})
print(with_context_response.content[:300] + "...")

---

## 2. Core Prompt Engineering Types

Let's explore the main types of prompt engineering techniques with simple, clear examples.

### 2.1 Zero-Shot Prompting

**What it is**: Asking the AI to perform a task without providing any examples.

**When to use**: For straightforward tasks where the AI already understands what you want.

In [ ]:
# Zero-shot example: Sentiment analysis
zero_shot_template = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analyzer. Classify the sentiment of the given text as POSITIVE, NEGATIVE, or NEUTRAL."),
    ("human", "Text: {text}\n\nSentiment:")
])

# Test with different texts
test_texts = [
    "I absolutely love this new restaurant! The food is amazing.",
    "The weather today is okay, nothing special.",
    "I hate waiting in long lines. This is frustrating."
]

zero_shot_chain = zero_shot_template | llm

print("=== ZERO-SHOT SENTIMENT ANALYSIS ===")
for i, text in enumerate(test_texts, 1):
    response = zero_shot_chain.invoke({"text": text})
    print(f"{i}. Text: '{text}'")
    print(f"   Sentiment: {response.content}\n")

### 2.2 One-Shot Prompting

**What it is**: Providing exactly one example to show the AI what you want.

**When to use**: When you need to demonstrate a specific format or style.

In [ ]:
# One-shot example: Product description generation
one_shot_template = ChatPromptTemplate.from_messages([
    ("system", "You generate engaging product descriptions for e-commerce. Follow the example format."),
    ("human", "Product: Wireless Headphones\nFeatures: Noise-cancelling, 30-hour battery, Bluetooth 5.0"),
    ("ai", "🎧 **Premium Wireless Headphones** 🎧\n\n✨ Experience crystal-clear audio with advanced noise-cancelling technology\n🔋 All-day listening with impressive 30-hour battery life\n📱 Seamless connectivity with latest Bluetooth 5.0\n\nPerfect for music lovers and professionals alike!"),
    ("human", "Product: {product}\nFeatures: {features}")
])

# Test the one-shot approach
one_shot_chain = one_shot_template | llm
response = one_shot_chain.invoke({
    "product": "Smartwatch",
    "features": "Heart rate monitoring, GPS tracking, Water-resistant, 7-day battery"
})

print("=== ONE-SHOT PRODUCT DESCRIPTION ===")
print(response.content)

### 2.3 Few-Shot Prompting

**What it is**: Providing multiple examples to help the AI learn a pattern.

**When to use**: For complex tasks where you need consistent formatting or when the task isn't obvious.

In [ ]:
# Few-shot example: Email categorization
examples = [
    {
        "email": "Your order #12345 has been shipped and will arrive tomorrow.",
        "category": "ORDER_UPDATE"
    },
    {
        "email": "Don't miss our 50% off sale this weekend only!",
        "category": "PROMOTION"
    },
    {
        "email": "We're sorry to hear about your recent experience. Please reply with details.",
        "category": "CUSTOMER_SERVICE"
    }
]

# Create few-shot template
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Email: {email}"),
    ("ai", "Category: {category}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You categorize emails into: ORDER_UPDATE, PROMOTION, CUSTOMER_SERVICE, or OTHER"),
    few_shot_prompt,
    ("human", "Email: {email}")
])

# Test few-shot learning
few_shot_chain = final_prompt | llm
test_email = "Thank you for your feedback! We've updated our privacy policy."

response = few_shot_chain.invoke({"email": test_email})

print("=== FEW-SHOT EMAIL CATEGORIZATION ===")
print(f"Test Email: '{test_email}'")
print(f"Result: {response.content}")

### 2.4 Chain-of-Thought Prompting

**What it is**: Asking the AI to think step-by-step and show its reasoning.

**When to use**: For complex problems that require logical reasoning or multi-step solutions.

In [ ]:
# Chain-of-thought example: Math word problems
cot_template = ChatPromptTemplate.from_messages([
    ("system", "You solve math problems step by step. Always show your reasoning clearly."),
    ("human", """Let's work through this step by step.

Problem: {problem}

Please think through this step by step:
1. What information do we have?
2. What do we need to find?
3. What steps do we need to take?
4. Show the calculations
5. State the final answer
""")
])

# Test with a word problem
math_problem = "A store sells books for $15 each. If they offer a 20% discount for buying 5 or more books, how much would it cost to buy 8 books?"

cot_chain = cot_template | llm
response = cot_chain.invoke({"problem": math_problem})

print("=== CHAIN-OF-THOUGHT PROBLEM SOLVING ===")
print(response.content)

### 2.5 Role-Based Prompting

**What it is**: Assigning the AI a specific role or persona to guide its responses.

**When to use**: When you need expertise from a specific perspective or want a particular communication style.

In [ ]:
# Role-based prompting examples
roles = {
    "teacher": "You are an elementary school teacher explaining concepts to 8-year-old students. Use simple words and fun examples.",
    "scientist": "You are a research scientist. Provide accurate, technical explanations with scientific terminology.",
    "comedian": "You are a friendly comedian. Make your explanations funny and entertaining while staying informative."
}

def create_role_prompt(role_description):
    return ChatPromptTemplate.from_messages([
        ("system", role_description),
        ("human", "{question}")
    ])

question = "Why do we have seasons?"

print("=== ROLE-BASED PROMPTING COMPARISON ===")
print(f"Question: {question}\n")

for role_name, role_desc in roles.items():
    role_template = create_role_prompt(role_desc)
    role_chain = role_template | llm
    response = role_chain.invoke({"question": question})
    
    print(f"=== {role_name.upper()} PERSPECTIVE ===")
    print(response.content[:300] + "...\n")

---

## 3. Simple Practical Examples

Let's apply these techniques to common real-world tasks.

### 3.1 Text Classification

In [ ]:
# News article classification
classification_template = ChatPromptTemplate.from_messages([
    ("system", """Classify news articles into categories: TECHNOLOGY, SPORTS, POLITICS, BUSINESS, HEALTH, ENTERTAINMENT.
    
    Output format: CATEGORY - Brief reason"""),
    ("human", "Article headline: {headline}\n\nClassify this article:")
])

headlines = [
    "New iPhone Release Features Revolutionary Camera Technology",
    "Local Basketball Team Wins Championship After 20-Year Drought",
    "Stock Market Reaches New High Amid Economic Recovery",
    "Scientists Discover New Treatment for Common Cold"
]

classification_chain = classification_template | llm

print("=== NEWS ARTICLE CLASSIFICATION ===")
for i, headline in enumerate(headlines, 1):
    response = classification_chain.invoke({"headline": headline})
    print(f"{i}. '{headline}'")
    print(f"   → {response.content}\n")

### 3.2 Information Extraction

In [ ]:
# Extract structured information from text
extraction_template = ChatPromptTemplate.from_messages([
    ("system", """Extract key information from the text and format as JSON with these fields:
    {
        "name": "person's name",
        "age": "age if mentioned",
        "location": "city/state",
        "occupation": "job title",
        "interests": ["list", "of", "hobbies"]
    }
    
    Use "unknown" for missing information."""),
    ("human", "Text: {text}")
])

sample_text = """Meet Sarah Johnson, a 28-year-old software engineer living in Austin, Texas. 
She loves hiking, photography, and playing guitar in her spare time. Sarah has been 
coding for over 5 years and recently started her own tech startup."""

extraction_chain = extraction_template | llm
response = extraction_chain.invoke({"text": sample_text})

print("=== INFORMATION EXTRACTION ===")
print(f"Original Text: {sample_text}\n")
print(f"Extracted Information:")
print(response.content)

### 3.3 Content Generation

In [ ]:
# Generate social media posts
content_generation_template = ChatPromptTemplate.from_messages([
    ("system", """You create engaging social media posts. Include:
    - Catchy opening
    - Key message
    - Call to action
    - Relevant hashtags
    
    Keep it under 280 characters for Twitter-style posts."""),
    ("human", """Topic: {topic}
    Target audience: {audience}
    Tone: {tone}
    
    Create a social media post:""")
])

content_requests = [
    {
        "topic": "New coffee shop opening",
        "audience": "Local coffee lovers",
        "tone": "Excited and welcoming"
    },
    {
        "topic": "Tips for working from home",
        "audience": "Remote workers",
        "tone": "Helpful and professional"
    }
]

content_chain = content_generation_template | llm

print("=== CONTENT GENERATION ===")
for i, request in enumerate(content_requests, 1):
    response = content_chain.invoke(request)
    print(f"Post {i} - {request['topic']}:")
    print(response.content)
    print(f"Characters: {len(response.content)}\n")

### 3.4 Question Answering with Different Approaches

In [ ]:
# Compare different QA approaches
context = """Python is a high-level programming language created by Guido van Rossum 
and first released in 1991. It's known for its simple, readable syntax that 
emphasizes code readability. Python supports multiple programming paradigms 
including procedural, object-oriented, and functional programming. It's widely 
used in web development, data science, artificial intelligence, and automation."""

question = "What is Python used for?"

# Approach 1: Direct QA
direct_qa = ChatPromptTemplate.from_messages([
    ("human", "Question: {question}")
])

# Approach 2: Context-based QA
context_qa = ChatPromptTemplate.from_messages([
    ("human", "Context: {context}\n\nQuestion: {question}\n\nAnswer based on the context:")
])

# Approach 3: Step-by-step QA
step_qa = ChatPromptTemplate.from_messages([
    ("human", """Context: {context}
    
    Question: {question}
    
    Please:
    1. Identify relevant information from the context
    2. Provide a clear answer
    3. Explain your reasoning""")
])

approaches = [
    ("Direct QA (no context)", direct_qa, {"question": question}),
    ("Context-based QA", context_qa, {"context": context, "question": question}),
    ("Step-by-step QA", step_qa, {"context": context, "question": question})
]

print("=== QUESTION ANSWERING COMPARISON ===")
print(f"Question: {question}\n")

for approach_name, template, inputs in approaches:
    chain = template | llm
    response = chain.invoke(inputs)
    print(f"=== {approach_name} ===")
    print(response.content[:250] + "...\n")

---

## 4. Prompt Optimization Techniques

Learn how to make your prompts more effective.

### 4.1 Instruction Clarity: Clear vs Vague

In [ ]:
# Compare vague vs clear instructions
vague_prompt = ChatPromptTemplate.from_messages([
    ("human", "Write about {topic}")
])

clear_prompt = ChatPromptTemplate.from_messages([
    ("human", """Write a 150-word informative paragraph about {topic} that:
    1. Defines what it is
    2. Explains why it's important
    3. Gives 2 practical examples
    4. Uses simple language suitable for beginners
    
    Format: One cohesive paragraph, no bullet points.""")
])

topic = "machine learning"

print("=== INSTRUCTION CLARITY COMPARISON ===")
print(f"Topic: {topic}\n")

# Vague version
vague_chain = vague_prompt | llm
vague_response = vague_chain.invoke({"topic": topic})
print("=== VAGUE INSTRUCTIONS ===")
print(vague_response.content[:300] + "...\n")

# Clear version
clear_chain = clear_prompt | llm
clear_response = clear_chain.invoke({"topic": topic})
print("=== CLEAR INSTRUCTIONS ===")
print(clear_response.content)

### 4.2 Output Formatting

In [ ]:
# Structured output formatting
formatting_template = ChatPromptTemplate.from_messages([
    ("system", "You analyze products and provide structured reviews."),
    ("human", """Product: {product}
    Price: {price}
    Features: {features}
    
    Provide a review in this exact format:
    
    ## Product Review: [Product Name]
    
    **Rating:** [1-5 stars] ⭐
    **Price:** [price evaluation - expensive/reasonable/budget]
    
    **Pros:**
    • [pro 1]
    • [pro 2]
    
    **Cons:**
    • [con 1]
    • [con 2]
    
    **Best For:** [target user]
    
    **Bottom Line:** [one sentence summary]""")
])

product_info = {
    "product": "Wireless Gaming Mouse",
    "price": "$79.99",
    "features": "RGB lighting, 16000 DPI, 70-hour battery, ergonomic design"
}

formatting_chain = formatting_template | llm
response = formatting_chain.invoke(product_info)

print("=== STRUCTURED OUTPUT FORMATTING ===")
print(response.content)

### 4.3 Temperature Control: Creativity vs Consistency

In [ ]:
# Test different temperature settings
creative_prompt = ChatPromptTemplate.from_messages([
    ("human", "Write a creative opening sentence for a story about {scenario}")
])

scenario = "a robot discovering emotions"

# Different temperature models
conservative_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)  # Low creativity, high consistency
balanced_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)     # Balanced
creative_llm = ChatOpenAI(model="gpt-4o-mini", temperature=1.0)     # High creativity, low consistency

models = [
    ("Conservative (temp=0.1)", conservative_llm),
    ("Balanced (temp=0.7)", balanced_llm),
    ("Creative (temp=1.0)", creative_llm)
]

print("=== TEMPERATURE COMPARISON ===")
print(f"Scenario: {scenario}\n")

for model_name, model in models:
    chain = creative_prompt | model
    response = chain.invoke({"scenario": scenario})
    print(f"=== {model_name} ===")
    print(response.content)
    print()

---

## 5. Common Patterns and Templates

Reusable patterns for common tasks.

### 5.1 The STAR Pattern (Situation, Task, Action, Result)

In [ ]:
# STAR pattern for structured responses
star_template = ChatPromptTemplate.from_messages([
    ("system", """You provide structured responses using the STAR method:
    - Situation: Set the context
    - Task: Describe what needed to be done
    - Action: Explain the steps taken
    - Result: Share the outcome
    
    Format each section clearly with headers."""),
    ("human", "Describe how to handle this scenario using the STAR method: {scenario}")
])

scenario = "A team project is behind schedule with conflicting opinions on priorities"

star_chain = star_template | llm
response = star_chain.invoke({"scenario": scenario})

print("=== STAR PATTERN EXAMPLE ===")
print(response.content)

### 5.2 Multi-Step Problem Solving Template

In [ ]:
# Multi-step problem solving
problem_solving_template = ChatPromptTemplate.from_messages([
    ("system", """You solve problems systematically. Always follow this structure:
    
    🔍 **ANALYZE THE PROBLEM**
    - What is the main issue?
    - What are the constraints?
    - What resources are available?
    
    🎯 **IDENTIFY SOLUTIONS**
    - List 2-3 possible approaches
    - Consider pros and cons of each
    
    ⚡ **RECOMMENDED ACTION**
    - Choose the best approach
    - Provide step-by-step implementation
    
    📊 **MEASURE SUCCESS**
    - How will you know it worked?
    - What metrics to track?
    """),
    ("human", "Problem: {problem}")
])

problem = "Our website loads slowly on mobile devices, causing users to leave"

problem_solving_chain = problem_solving_template | llm
response = problem_solving_chain.invoke({"problem": problem})

print("=== MULTI-STEP PROBLEM SOLVING ===")
print(response.content)

### 5.3 Conditional Logic Template

In [ ]:
# Template with conditional responses
conditional_template = ChatPromptTemplate.from_messages([
    ("system", """You provide personalized fitness advice based on experience level:
    
    - BEGINNER (0-6 months): Focus on form, basic exercises, 2-3 days/week
    - INTERMEDIATE (6 months - 2 years): Progressive overload, 3-4 days/week
    - ADVANCED (2+ years): Complex movements, specialized programs, 4-6 days/week
    
    Tailor your advice to their level. Include specific exercises and rep ranges."""),
    ("human", """User Profile:
    - Experience level: {level}
    - Goal: {goal}
    - Available time: {time_available}
    - Equipment: {equipment}
    
    Provide a personalized fitness plan.""")
])

user_profiles = [
    {
        "level": "BEGINNER",
        "goal": "lose weight",
        "time_available": "30 minutes, 3 days/week",
        "equipment": "none (bodyweight only)"
    },
    {
        "level": "INTERMEDIATE",
        "goal": "build muscle",
        "time_available": "45 minutes, 4 days/week",
        "equipment": "full gym access"
    }
]

conditional_chain = conditional_template | llm

print("=== CONDITIONAL LOGIC TEMPLATES ===")
for i, profile in enumerate(user_profiles, 1):
    response = conditional_chain.invoke(profile)
    print(f"=== User {i}: {profile['level']} ===")
    print(response.content[:400] + "...\n")

### 5.4 Error Handling and Validation

In [ ]:
# Template with input validation
validation_template = ChatPromptTemplate.from_messages([
    ("system", """You help with recipe calculations. Before calculating:
    
    1. CHECK if the input makes sense:
       - Is the number of servings reasonable (1-50)?
       - Are ingredients listed?
       - Is the original serving size mentioned?
    
    2. If input is invalid, politely ask for clarification
    3. If valid, provide scaled recipe with clear measurements
    
    Always double-check your math!"""),
    ("human", """Original recipe serves: {original_servings}
    Ingredients: {ingredients}
    Need to serve: {target_servings}
    
    Please scale this recipe.""")
])

# Test with valid and invalid inputs
test_cases = [
    {
        "original_servings": "4",
        "ingredients": "2 cups flour, 1 cup sugar, 3 eggs, 1/2 cup butter",
        "target_servings": "8"
    },
    {
        "original_servings": "unclear",
        "ingredients": "some flour, sugar",
        "target_servings": "100"
    }
]

validation_chain = validation_template | llm

print("=== INPUT VALIDATION EXAMPLE ===")
for i, case in enumerate(test_cases, 1):
    response = validation_chain.invoke(case)
    print(f"=== Test Case {i} ===")
    print(f"Input: Serves {case['original_servings']} → {case['target_servings']}")
    print(f"Response: {response.content[:300]}...\n")

---

## 6. Interactive Examples with LangChain

Advanced LangChain features for prompt engineering.

### 6.1 Prompt Comparison Tool

In [ ]:
# Tool to compare different prompt approaches
class PromptTester:
    def __init__(self, llm):
        self.llm = llm
    
    def compare_prompts(self, prompts_dict, input_data):
        results = {}
        
        for name, template in prompts_dict.items():
            chain = template | self.llm
            response = chain.invoke(input_data)
            results[name] = {
                'response': response.content,
                'length': len(response.content),
                'template': str(template)
            }
        
        return results
    
    def print_comparison(self, results):
        print("=== PROMPT COMPARISON RESULTS ===")
        for name, result in results.items():
            print(f"\n=== {name} ===")
            print(f"Length: {result['length']} characters")
            print(f"Response: {result['response'][:200]}...")

# Create different prompt versions
prompt_versions = {
    "Basic": ChatPromptTemplate.from_messages([
        ("human", "Explain {concept}")
    ]),
    
    "Detailed": ChatPromptTemplate.from_messages([
        ("human", "Explain {concept} in simple terms with an example and why it's useful")
    ]),
    
    "Structured": ChatPromptTemplate.from_messages([
        ("system", "Provide explanations in this format: Definition, Example, Benefits, Use Cases"),
        ("human", "Explain: {concept}")
    ])
}

# Test the prompts
tester = PromptTester(llm)
results = tester.compare_prompts(prompt_versions, {"concept": "blockchain technology"})
tester.print_comparison(results)

### 6.2 Dynamic Prompt Builder

In [ ]:
# Dynamic prompt builder based on user needs
class DynamicPromptBuilder:
    def __init__(self):
        self.base_system = "You are a helpful assistant."
        self.components = {
            'role': {
                'teacher': "You are an experienced teacher who explains concepts clearly.",
                'expert': "You are a subject matter expert providing detailed technical information.",
                'friend': "You are a knowledgeable friend giving casual, friendly advice."
            },
            'format': {
                'bullet': "Format your response as bullet points.",
                'numbered': "Format your response as numbered steps.",
                'paragraph': "Format your response as flowing paragraphs."
            },
            'length': {
                'brief': "Keep your response under 100 words.",
                'moderate': "Aim for 150-300 words.",
                'detailed': "Provide a comprehensive response."
            }
        }
    
    def build_prompt(self, role=None, format_type=None, length=None, include_examples=False):
        system_parts = []
        
        if role:
            system_parts.append(self.components['role'].get(role, self.base_system))
        
        if format_type:
            system_parts.append(self.components['format'].get(format_type, ""))
        
        if length:
            system_parts.append(self.components['length'].get(length, ""))
        
        if include_examples:
            system_parts.append("Include practical examples in your response.")
        
        system_message = " ".join(filter(None, system_parts))
        
        return ChatPromptTemplate.from_messages([
            ("system", system_message),
            ("human", "{question}")
        ])

# Test dynamic prompt building
builder = DynamicPromptBuilder()

configurations = [
    {"role": "teacher", "format_type": "numbered", "length": "brief", "name": "Teacher Style"},
    {"role": "expert", "format_type": "paragraph", "length": "detailed", "include_examples": True, "name": "Expert Style"},
    {"role": "friend", "format_type": "bullet", "length": "moderate", "name": "Friendly Style"}
]

question = "How does photosynthesis work?"

print("=== DYNAMIC PROMPT BUILDER ===")
print(f"Question: {question}\n")

for config in configurations:
    name = config.pop('name')  # Remove name from config
    prompt = builder.build_prompt(**config)
    chain = prompt | llm
    response = chain.invoke({"question": question})
    
    print(f"=== {name} ===")
    print(response.content[:250] + "...\n")

### 6.3 Prompt Performance Metrics

In [ ]:
# Simple metrics for prompt effectiveness
import time
import re

class PromptMetrics:
    def __init__(self, llm):
        self.llm = llm
    
    def analyze_prompt(self, template, input_data, runs=3):
        chain = template | self.llm
        responses = []
        times = []
        
        for _ in range(runs):
            start_time = time.time()
            response = chain.invoke(input_data)
            end_time = time.time()
            
            responses.append(response.content)
            times.append(end_time - start_time)
        
        # Calculate metrics
        avg_length = sum(len(r) for r in responses) / len(responses)
        avg_time = sum(times) / len(times)
        consistency = self._calculate_consistency(responses)
        readability = self._calculate_readability(responses[0])
        
        return {
            'average_length': avg_length,
            'average_time': avg_time,
            'consistency': consistency,
            'readability_score': readability,
            'sample_response': responses[0]
        }
    
    def _calculate_consistency(self, responses):
        # Simple consistency check based on length variation
        lengths = [len(r) for r in responses]
        if not lengths:
            return 0
        avg = sum(lengths) / len(lengths)
        variance = sum((l - avg) ** 2 for l in lengths) / len(lengths)
        return max(0, 100 - (variance ** 0.5))
    
    def _calculate_readability(self, text):
        # Simple readability score (higher = more readable)
        sentences = len(re.split(r'[.!?]+', text))
        words = len(text.split())
        if sentences == 0:
            return 0
        avg_words_per_sentence = words / sentences
        # Prefer 10-20 words per sentence
        optimal_range = (10, 20)
        if optimal_range[0] <= avg_words_per_sentence <= optimal_range[1]:
            return 100
        else:
            distance = min(abs(avg_words_per_sentence - optimal_range[0]),
                         abs(avg_words_per_sentence - optimal_range[1]))
            return max(0, 100 - distance * 5)

# Test prompt metrics
metrics = PromptMetrics(llm)

test_prompts = {
    "Simple": ChatPromptTemplate.from_messages([
        ("human", "What is {topic}?")
    ]),
    "Detailed": ChatPromptTemplate.from_messages([
        ("system", "Provide clear, concise explanations suitable for beginners."),
        ("human", "Explain {topic} in simple terms with one example.")
    ])
}

topic = "artificial intelligence"

print("=== PROMPT PERFORMANCE METRICS ===")
for name, template in test_prompts.items():
    results = metrics.analyze_prompt(template, {"topic": topic})
    
    print(f"\n=== {name} Prompt ===")
    print(f"Average Length: {results['average_length']:.1f} characters")
    print(f"Average Time: {results['average_time']:.2f} seconds")
    print(f"Consistency Score: {results['consistency']:.1f}/100")
    print(f"Readability Score: {results['readability_score']:.1f}/100")
    print(f"Sample: {results['sample_response'][:150]}...")

---

## Summary and Key Takeaways

### 🎯 **Main Prompt Engineering Types**:

1. **Zero-Shot**: Direct task execution without examples
   - Best for: Simple, well-defined tasks
   - Example: "Classify this sentiment as positive, negative, or neutral"

2. **One-Shot**: Single example to guide behavior
   - Best for: Demonstrating specific format or style
   - Example: Show one product description, then ask for another

3. **Few-Shot**: Multiple examples for pattern learning
   - Best for: Complex formatting or classification tasks
   - Example: Show 3 email categorization examples, then classify new emails

4. **Chain-of-Thought**: Step-by-step reasoning
   - Best for: Complex problem-solving, math, logical reasoning
   - Example: "Let's solve this step by step: 1) What do we know? 2) What formula applies?"

5. **Role-Based**: Assign specific persona or expertise
   - Best for: Specialized knowledge or communication style
   - Example: "You are a pediatric nurse explaining vaccines to worried parents"

### 🚀 **Optimization Tips**:

- **Be Specific**: Clear instructions beat vague requests every time
- **Provide Context**: Help the AI understand the situation
- **Format Output**: Specify exactly how you want the response structured
- **Control Temperature**: Low for consistency, high for creativity
- **Test and Iterate**: Try different approaches and measure results

### 🛠️ **Practical Applications**:

- **Content Creation**: Blogs, social media, marketing copy
- **Data Analysis**: Classification, extraction, summarization
- **Customer Service**: FAQ responses, support tickets
- **Education**: Explanations, tutorials, assessments
- **Decision Support**: Analysis, recommendations, comparisons

### 📈 **Next Steps**:

1. **Practice**: Try each prompt type with your own use cases
2. **Measure**: Track what works best for your specific needs
3. **Build Templates**: Create reusable prompts for common tasks
4. **Combine Techniques**: Mix different approaches for complex scenarios
5. **Stay Updated**: Prompt engineering evolves with new models and techniques

---

**Remember**: Good prompt engineering is part art, part science. Start simple, be specific, and always test your results! 🎉